In [0]:
!git clone https://github.com/16cs009/alyceproject.git
import os
os.chdir("/content/alyceproject")

!pip install "python-socketio[client]"
!pip install -q gdown Unidecode==0.4.20 git+git://github.com/bootphon/phonemizer@master localimport
!apt-get install -y espeak
!pip install -q -r ./WaveRNN/requirements.txt

from os.path import exists, join, basename, splitext
tts_pretrained_model = 'tts_models/checkpoint_261000.pth.tar'
if not exists(tts_pretrained_model):
  !gdown -O {tts_pretrained_model} https://drive.google.com/uc?id=1otOqpixEsHf7SbOZIcttv3O7pG0EadDx
!mkdir ./public/audio

In [0]:
import wavernn_wrapper
import hashlib
WaveRNN_libraries = wavernn_wrapper.initializeWaveRNN()

def phaseIII_processStory(storylist):
	# Returns a list of list with the following as the content of list inside the list :>
	# 1. File name of the audio file generated by the server for corresponding entry in storylist.
	#	 (In case of NEWLINE the entry will be empty)
	audioOutputList = []
	pathToStaticFolder = "./public/audio/"
	for content in storylist:
		print("-->",content)
		if content[1]=="NEWLINE":
			audioOutputList.append("")
		else:
			filename = hashlib.sha224(content[0].encode()).hexdigest()+".wav"
			if content[1]=="DIALOGUE":	
				filename = hashlib.sha224(content[5].encode()).hexdigest()+".wav"
			if exists(pathToStaticFolder+filename):
				audioOutputList.append(filename)
			else:
				speech = None
				if content[1]=="DIALOGUE":	
					speech = wavernn_wrapper.textToSpeech(content[5],WaveRNN_libraries)
				else:
					speech = wavernn_wrapper.textToSpeech(content[0],WaveRNN_libraries)
				WaveRNN_libraries["audioprocessor"].save_wav(speech,pathToStaticFolder+filename)
				audioOutputList.append(filename)
	return audioOutputList

In [0]:
import socketio
import json
import time
import base64

sio = socketio.Client()
pathToStaticFolder = "./public/audio/"

def workHandler(input_data):
	sio.emit('phase-3-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "STARTED", "output": json.dumps([["None"]])})
	output_data = phaseIII_processStory(input_data["input"])
	output_to_server = []#output_data
	for filename in output_data:
		if len(filename)>0:
			f1 = open(pathToStaticFolder+filename,"rb")
			data = f1.read()
			encoded = str(base64.b64encode(data))[2:][:-1]
			f1.close()
			output_to_server.append(filename)
			sio.emit('phase-3-server-to-main-server-file-ready',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETED", "filename": filename, "output": json.dumps(encoded)})
		else:
			#sio.emit('phase-3-server-to-main-server-save-file',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETEDINTER", "output": ""})
			output_to_server.append("")
	#sio.sleep(30);
	sio.emit('phase-3-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETED", "output": json.dumps(output_to_server)})
	print("Phase 3 Completed!...")

@sio.on('main-server-to-phase-3-server')
def on_phaseThreeData(data):
	print("RECEIVED DATA : {}".format(data))
	sio.start_background_task(workHandler, data)

print("Starting Sub server of Phase 3 !...")
sio.connect(input("Enter NGROK Link : "))
sio.emit('server-ready',{"phase": 3})
print("Started Sub server of Phase 3 !...")

try:
	while True:
		time.sleep(1)
except KeyboardInterrupt:
	print("Interrupt Received!...")
	sio.disconnect()

print("Exiting Sub Server!....")